# Simulation

In [1]:
from blackjacksim.simulations import Game
from tqdm import tnrange
import numpy as np

S = None
for _ in tnrange(200):
    g = Game()
    for _ in range(100):
        g.round()
    if S is None:
        S = np.stack(g.data.State) 
        w = np.array(g.data.Advantage)
    else:
        S = np.concatenate([S, np.stack(g.data.State)])
        w = np.concatenate([w, np.stack(g.data.Advantage)])
        
S = np.stack(g.data.State)
print(S.shape)
i = np.array(g.data.Advantage.apply(np.sign))
i[i==0] = -1

print(i)




(100, 10)
[ 1.  1. -1.  1. -1. -1. -1.  1.  1. -1.  1.  1. -1. -1.  1. -1.  1.  1.
 -1.  1.  1. -1.  1. -1. -1. -1.  1.  1. -1. -1.  1.  1. -1.  1. -1.  1.
 -1. -1.  1.  1.  1. -1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1. -1. -1.
 -1. -1.  1.  1. -1. -1. -1. -1. -1.  1. -1.  1. -1.  1.  1.  1.  1.  1.
 -1. -1.  1. -1. -1.  1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.
  1. -1.  1.  1.  1.  1.  1. -1.  1.  1.]


In [1]:
from blackjacksim.simulations import Game
import matplotlib
%matplotlib inline
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from jupyterthemes import jtplot
jtplot.style(context='poster', fscale=1.4, spines=False, gridlines='--')

from blackjacksim.data import DefaultGameConfig
_def_conf = DefaultGameConfig()
def config(house_rules):
    _def_conf['house']['class'] = house_rules
    return _def_conf

try:
    df = df
except:
    df = None
pbar = tqdm(['Blackjack32', 'Blackjack65', 'Blackjack32NoSplit', 'Blackjack65NoSplit'])
trials = 200
rounds = 100
for house in pbar:
    for i in range(trials):
        pbar.set_description("{} {:04d}/{:04d}: ".format(house,i,trials-1))
        g = Game(config(house))
        for _ in range(rounds):
            g.round()
        if df is None:
            df = g.data 
        else:
            df = pd.concat([df,g.data]) 
            
sns.lineplot(x='Round', y='Pool', hue='House', data=df)
plt.show()

KeyboardInterrupt: 

In [35]:
from blackjacksim.pytorch import BlackjackDataset
from tqdm import tqdm_notebook as tqdm
from blackjacksim.data import DefaultGameConfig
_def_conf = DefaultGameConfig()
def config(size, cut_card_position):
    _def_conf['shoe']['params']['size'] = size
    _def_conf['shoe']['params']['cut_card_position'] = cut_card_position
    return _def_conf


b = BlackjackDataset(batch_size=10, config=config(1,.75), max_len=5)

for S,w in tqdm(b):
    print(S.sum(1).min(), S.sum(1).mean(), S.sum(1).max())

tensor(13.) tensor(33.1000) tensor(47.)
tensor(14.) tensor(30.1000) tensor(47.)
tensor(13.) tensor(30.3000) tensor(47.)
tensor(14.) tensor(33.7000) tensor(48.)
tensor(16.) tensor(30.4000) tensor(47.)



In [7]:
from blackjacksim.data import BasicStrategy
import numpy as np

basic = BasicStrategy()
action_to_class = {'Hit':[1,0,0,0],'Stand':[0,1,0,0],'Split':[0,0,1,0],'Double':[0.0,0,0,1]}
A = {'Hard':[], 'Soft':[], 'Splittable':[]}
b = {'Hard':[], 'Soft':[], 'Splittable':[]}
for dealer_up , v1 in basic.items():
    for hand_type, v2 in v1.items():
        for hand_value, action in v2.items():
            if hand_value == 'AA':
                hand_value = 22
            
            hand_value = int(hand_value)
            A[hand_type].append([int(dealer_up), int(hand_value)])
            #if action == 'Double':
                #action = 'Hit'
            b[hand_type].append(action_to_class[action])


# Naive classification without bias
for _type in ['Hard', 'Soft', 'Splittable']:
    _A = np.stack(A[_type])
    _b = np.stack(b[_type])
    Ai = np.linalg.pinv(_A)
    x = Ai@_b
    acc = (np.argmax(_A@x,1) == np.argmax(_b,1)).mean()
    print(acc)
    
def stable_softmax(X):
    exps = np.exp(X - np.max(X,1,keepdims=True))
    return exps / np.sum(exps,1, keepdims=True)

# Naive classification with bias
lab_to_class = list(action_to_class.keys())
acc_f = lambda f: (pred[f] == lab[f]).mean()
for _type in ['Hard', 'Soft', 'Splittable']:
    _A = np.stack(A[_type])
    _b = np.stack(b[_type])
    _A = np.concatenate([_A, np.ones((_A.shape[0],1))],1)
    Ai = np.linalg.pinv(_A)
    x = Ai@_b
    out = _A@x
    pred = np.argmax(out,1)
    lab = np.argmax(_b,1)
    acc = (pred == lab).mean()
    print(acc)
    wr = pred != lab
    print('Hit',out[lab==0].mean(0), acc_f(lab==0))
    print('Stand', out[lab==1].mean(0), acc_f(lab==1))
    print('Split', out[lab==2].mean(0), acc_f(lab==2))
    print('Double', out[lab==3].mean(0), acc_f(lab==3))
    #for o,s,p,l in zip(out[wr], _A[wr], pred[wr], lab[wr]):
        #print(o,s[:-1],lab_to_class[p],lab_to_class[l])

    

0.7555555555555555
0.52
0.6
0.8055555555555556
Hit [0.76147536 0.09787325 0.         0.14065138] 0.8604651162790697
Stand [0.11530274 0.80654249 0.         0.07815477] 0.9726027397260274
Split [nan nan nan nan] nan
Double [0.57600091 0.27168086 0.         0.15231823] 0.0
0.75
Hit [0.71016763 0.0722695  0.         0.21756286] 0.8936170212765957
Stand [0.10292929 0.77064279 0.         0.12642792] 1.0
Split [nan nan nan nan] nan
Double [0.51127273 0.20860606 0.         0.28012121] 0.0
0.66
Hit [0.50728485 0.0426303  0.31190303 0.13818182] 0.76
Stand [0.08198135 0.30515152 0.57230769 0.04055944] 0.07692307692307693
Split [0.14995338 0.14307692 0.61571096 0.09125874] 0.8846153846153846
Double [0.34545455 0.05272727 0.47454545 0.12727273] 0.0


/home/charlie/.virtualenvs/blackjacksim/lib/python3.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: Mean of empty slice.
/home/charlie/.virtualenvs/blackjacksim/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: Mean of empty slice.


In [124]:
from blackjacksim.entities import Deck, Hand
from blackjacksim.strategies import basic
import itertools
import numpy as np
import pandas as pd

action_to_class = {'Hit':[1,0,0,0],'Stand':[0,1,0,0],'Split':[0,0,1,0],'Double':[0.0,0,0,1]}

hands = [Hand(h) for h in itertools.product(Deck(),Deck())]
dups = Deck()

t = []
for hand, dup  in itertools.product(hands, dups):
    tup = tuple(c.value for c in (*hand,dup))
    c = (tup, basic(hand,dup))
    if c not in t:
        t.append(c)
print(len(t))
A = []
b = []
for  a, _b in t:
    A.append(a)
    b.append(action_to_class[_b])
    
A = np.stack(A)
b = np.array(b)

1001


In [125]:
A_ = np.concatenate([A, np.ones((A.shape[0],1))],1)
Ai = np.linalg.pinv(A_)
x = Ai@b
out = A_@x
pred = np.argmax(out,1)
lab = np.argmax(b,1)

lab_to_class = list(action_to_class.keys())
l2c = lambda x: lab_to_class[x]

df = pd.DataFrame({'Prediction':pred, 'Label':lab, 'Hand':[a[0:-1] for a in A], 'Up Card':[a[-1] for a in A]})
df['Label Name'] = df.Label.apply(l2c)
df['Prediction Name'] = df.Prediction.apply(l2c)
df['Correct'] = df.Prediction == df.Label

print('Accuracy: {:.2f}%\n'.format(df.Correct.mean()*100))
print(df.groupby('Label Name').Correct.mean().to_latex())
print(df.groupby('Prediction Name').Correct.mean().to_latex())

Accuracy: 65.53%

\begin{tabular}{lr}
\toprule
{} &   Correct \\
Label Name &           \\
\midrule
Double     &  0.036842 \\
Hit        &  0.828261 \\
Split      &  0.000000 \\
Stand      &  0.884488 \\
\bottomrule
\end{tabular}

\begin{tabular}{lr}
\toprule
{} &   Correct \\
Prediction Name &           \\
\midrule
Double          &  0.388889 \\
Hit             &  0.667250 \\
Stand           &  0.650485 \\
\bottomrule
\end{tabular}

